## 3. 음향 모델

음성인식에서 음성 데이터의 특징을 추출하는 데 사용되는 모델
음성 데이터를 시간에 따라 숨겨진 상태로 모델링하고, 이 상태를 사용하여 음성의 특징 추출

음향 모델 구성 요소

초기 상태 분포 - 음성 인식 시작 부분에서 음성 상태가 어떤 것일지 확률적으로 나타내는 분포

상태 전이 확률 - 한 상태에서 다음 상태로 전이할 확률을 나타내는 행렬

관측 확률  - 각 상태에서 관측될 수 있는 음성 신호의 확률을 나타내는 행렬


음향 모델 종류
HMM( Hidden Markov Model )
GMM( Gaussian Mixture Model )
GMM-HMM
DNN-HMM


#HMM(Hidden Markov Model)

HMM은 Markov Chain을  전제로 한 모델

Markov Chain
    마코프 성질을 가진 이산 확률 과정
    마코프 성질 - 한 상태(state) qi가 나타날 확률은 단지 이전 상태 qi-q에만 의존한다.
        한 상태에서 다른 상태로의 전이는 그동안 상태 전이에 대한 긴이력을 필요로 하지 않고 바로 직전 상태로의 전이로 추정할 수 있다.
    
    마코프 체인은 시간 변화에 따른 상태들의 분포 추적에 관심을 갖는다.



## HMM

우도 계산의 이해

In [1]:
import random

import os
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

import soundfile as sf
import librosa
import librosa.display
import random

import IPython.display as idp

# 푸리에 변환을 위한 라이브러리
import math
from scipy.interpolate import splrep, splev

In [2]:
states = ['Fair', 'Loaded'] # 은닉상태
ovservations = [1,2,3,4,5,6] # 관측치 


In [3]:
#전이 확률 
transitions ={
    'Fair': {'Fair' : 0.95, 'Loaded':0.055},
    'Loaded': {"Fair":0.10,'Loaded':0.90}
}

In [4]:
#방출확률
emissions = {
    'Fair': {1: 1/6, 2: 1/6, 3: 1/6, 4 : 1/6, 5 :1/6, 6: 1/6},
    'Loaded' : {1: 1/10, 2: 1/10, 3: 1/10, 4: 1/10, 5: 1/10, 6: 5/10}
}

In [5]:
# 초기 상태 분포
initial_probs = {'Fair': 0.5, 'Loaded':0.5}

In [6]:
#관측 순서
observed_sequence = [3,5,2,4,6]

In [16]:

states = ['Fair', 'Loaded']  # 은닉상태
observations = [1, 2, 3, 4, 5, 6]  # 관측치

transitions = {
    'Fair': {'Fair': 0.95, 'Loaded': 0.055},
    'Loaded': {'Fair': 0.10, 'Loaded': 0.90}
}

# 방출확률
emissions = {
    'Fair': {1: 1/6, 2: 1/6, 3: 1/6, 4: 1/6, 5: 1/6, 6: 1/6},
    'Loaded': {1: 1/10, 2: 1/10, 3: 1/10, 4: 1/10, 5: 1/10, 6: 5/10}
}

# 초기 상태 분포
initial_probs = {'Fair': 0.5, 'Loaded': 0.5}

# 관측 순서
observed_sequence = [3, 5, 2, 4, 6]

# 우도 계산 (Forward Algorithm 사용)
def compute_likelihood(sequence):
    print(states)
    n_states = len(states)
    n_obs = len(sequence)

    alpha = np.zeros((n_obs, n_states))
    for i, obs in enumerate(sequence):
        for j, state in enumerate(states):
            if i == 0:
                alpha[i, j] = initial_probs[state] * emissions[state][obs]
            else:
                alpha[i, j] = sum(alpha[i-1, k] * transitions[states[k]][state] * emissions[state][obs] for k in range(n_states))

    likelihood = np.sum(alpha[-1])

    return likelihood

likelihood = compute_likelihood(observed_sequence)

print(f'Likelihood: {likelihood}')


['Fair', 'Loaded']
Likelihood: 9.730013278034976e-05


In [15]:
likelihood = compute_likelihood(observed_sequence)

print(f'Likelihood: {likelihood}')

UnboundLocalError: local variable 'states' referenced before assignment

## Foward Algorithm/ Backward Algorithm 이해

In [29]:
_states = ['hot', 'cold']#은닉 상태
symbols = ['1', '2', '3'] #관측치

In [18]:
#전이확률
trans_prob = {
    'hot': { 'hot' : 0.6, 'cold' : 0.4 },
    'cold': { 'hot' : 0.4, 'cold' : 0.6 }
}

In [19]:
# 방출확률
emit_prob = {
    'hot': { '1' : 0.2, '2' : 0.4, '3' : 0.4 },
    'cold': { '1' : 0.5, '2' : 0.4, '3' : 0.1 }
}

In [20]:
#초기 상태 분포
start_prob = {
    'hot' : 0.8,
    'cold' : 0.2
}

In [21]:
# 관측 순서
sequence = ['3', '1', '3']

## Forward 구현

In [43]:

def _forward(sequence):
    # sequence : O
    # 아이스크림 소비 기록 시퀀스 [3, 1, 3]
    sequence_length = len(sequence)
    if sequence_length == 0:
        return []
    
    # Dynamic Programming
    # 앞으로 중간 계산된 값들은 alpha라는 변수에 저장
    alpha = [{}]
    
    # 시작 지점의 alpha값 계산 후 alpha[0]에 저장
    # alpha[0] = {'hot' : p(hot\|start) * p(3\|hot), 
    #             'cold' : p(cold\|start) * p(3\|cold)}
    # p(3\|cold) : emit_prob('cold', 3)
    # sequence[0] : 3
    for state in _states:
        alpha[0][state] = start_prob[state] * emit_prob[state][sequence[0]]
    
    # sequence의 두번째 값부터 마지막까지 likelihood 모두 계산
    # index : 위 수식에서 t
    for index in range(1, sequence_length):
        alpha.append({})
        for state_to in _states:
            prob = 0
            for state_from in _states:
                # += : 위 수식에서 Σ
                # alpha[index-1] : 위 수식에서 α_t-1
                # state_from : 위 수식에서 i
                # state_to : 위 수식에서 j
                # trans_prob : 위 수식에서 a_ij
                prob += alpha[index - 1][state_from] * \
                    trans_prob[state_from][state_to]
            # emit_prob : 위 수식에서 b
            # sequence[index] : 위 수식에서 o_t 
            alpha[index][state_to] = prob * emit_prob[state_to][sequence[index]]

    return alpha

In [45]:
alpha =_forward(sequence)
print(alpha)

[{'hot': 0.32000000000000006, 'cold': 0.020000000000000004}, {'hot': 0.04000000000000001, 'cold': 0.07000000000000002}, {'hot': 0.02080000000000001, 'cold': 0.005800000000000001}]


## Backward 구현

In [46]:
def _backward(sequence):
        sequence_length = len(sequence)
        if sequence_length == 0:
            return []

        beta = [{}]
        for state in _states:
            beta[0][state] = 1

        for index in range(sequence_length - 1, 0, -1):
            beta.insert(0, {})
            for state_from in _states:
                prob = 0
                for state_to in _states:
                    prob += beta[1][state_to] * \
                        trans_prob[state_from][state_to] * \
                        emit_prob[state_to][sequence[index]]
                beta[0][state_from] = prob

        return beta

In [47]:
beta = _backward(sequence)
print(beta)

[{'hot': 0.07760000000000002, 'cold': 0.0884}, {'hot': 0.28, 'cold': 0.22000000000000003}, {'hot': 1, 'cold': 1}]


## DECODE Viterbi algorithm 구현

In [52]:
def decode(sequence):
    # sequence : O
    # sequence_length : T
    sequence_length = len(sequence)
    if sequence_length == 0:
        return []
    
    # delta : 비터비 확률 v
    # Dynamic Programming : 중간 계산값 저장해 활용
    delta = {}
    
    # 시작 지점의 delta값 계산
    for state in _states:
        # start_prob(state) : p(cold\|start) or p(hot\|start)
        # sequence[0] : 관측 시퀀스의 첫번째 요소, o_1, '3'
        # emit_prob(state, sequence[0]) : p(3\|cold) or p(3\|hot)
        delta[state] = start_prob[state] * emit_prob[state][sequence[0]]
        
    # pre : backtrace
    pre = []
    
    # sequence의 두번째 값부터 마지막까지 delta, backtrace 모두 계산
    # index : 위 수식에서 t
    for index in range(1, sequence_length):
        # delta_bar : t번째 관측치의 비터비 확률들
        # index가 거듭될수록 그 요소가 늘어남
        # 다 돌면 sequence_length 길이
        delta_bar = {}
        # pre_state : t번째 관측치의 backtrace들
        # index가 거듭될수록 그 요소가 늘어남
        # 다 돌면 sequence_length 길이
        pre_state = {}
        for state_to in _states:
            max_prob = 0
            max_state = None # backtrace 변수
            for state_from in _states:
                # state_from : 위 수식에서 i
                # state_to : 위 수식에서 j
                # delta[state_from] : 직전 상태의 비터비 확률(저장된 값 불러와 계산량 줄임)
                # trans_prob : 위 수식에서 a
                prob = delta[state_from] * trans_prob[state_from][state_to]
                # 비터비 확률 수식에서 i에 대해 최대값을 구하는데,
                # 방출확률 b는 i에 대해 무관하므로 최대값 연산에서 제외
                if prob > max_prob:
                    # 최대값 저장 : 현재 상태의 비터비 확률
                    max_prob = prob 
                    # 최대값의 위치 저장 : 현재 상태의 backtrace
                    max_state = state_from 
            delta_bar[state_to] = max_prob * emit_prob[state_to][sequence[index]]
            pre_state[state_to] = max_state
        # o_2까지의 비터비 확률을 구했다면 o_1 이전의 비터비 확률은 불필요
        # o_2의 비터비 확률들의 모음인 delta_bar를 전체 delta에 덮어씌움
        delta = delta_bar
        # o_2까지의 backtrace를 구했다 하더라도 o_3은 달라질 수 있음
        # pre에 pre_state를 append
        pre.append(pre_state)
    
    # 전체 시퀀스를 대상으로 최대 비터비확률과
    # 최대 비터비 확률을 내는 state 찾기
    # 현재 delta에는 시퀀스의 마지막 요소(O_T)에 
    # 해당하는 비터비 확률들이 저장돼 있기 때문
    # (state로만 구분되어 있음)
    max_state = None
    max_prob = 0
    for state in _states:
        if delta[state] > max_prob:
            max_prob = delta[state]
            max_state = state

    if max_state is None:
        return []
    
    # 최대 비터비 확률을 내는 state가 backtrace의 첫번째 요소
    result = [max_state]
    # index를 시퀀스의 역방향으로 후진하면서
    for index in range(sequence_length - 1, 0, -1):
        # index에 해당하는 max_state들을 뽑아내기
        # 이는 저 위쪽에서 이미 max_state들을 저장해두었기 때문에 가능
        max_state = pre[index - 1][max_state]
        # 뽑아낸 max_state들을 result의 첫번째 위치에 저장
        result.insert(0, max_state)

    return result

In [53]:
result = decode(sequence)
print(result)

['hot', 'cold', 'hot']
